In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install captum
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=4935d726da776e92597d708bfa67de64c9652685ea068631db2d0530bb011ddd
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=6045b3e17150401ce39ab572b3567040713be6b973e6195731d6474bdabfb837
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from captum.attr import visualization as viz
from fvcore.nn import FlopCountAnalysis, flop_count_table
from captum.attr import IntegratedGradients, LayerConductance, DeepLift, LayerDeepLift,LayerIntegratedGradients
import itertools
import pandas as pd
import os
import random
import numpy as np
from itertools import product
from torch.utils.data import SequentialSampler, RandomSampler
from torch.utils.data import DataLoader

In [4]:
method_names = ["LayerIntegratedGradients", "LayerDeepLift"]
INPUT_SHAPE= (1, 1, 28, 28)
Training_Device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Feature_Attribution_Device = torch.device("cpu")
NUM_EPOCHS=1
NUM_CLASSES=10
IN_CHANNELS=1
NUM_RUN=15

# Model structure defination

In [5]:
class Inception(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(Inception, self).__init__()

        # Define the 1x1 and 3x3 convolution module
        self.conv1x1_3x3 = nn.Sequential(
            nn.Conv2d(in_channels, 8, kernel_size=1),
            nn.Conv2d(8, 16, kernel_size=3, padding=1),
        )

        # First branch with 1x1 and 3x3 convolution
        self.branch1 = self.conv1x1_3x3

        # Second branch with 1x1 and 3x3 convolution
        self.branch2 = self.conv1x1_3x3

        # Third branch with 1x1 and 3x3 convolution
        self.branch3 = self.conv1x1_3x3

        # The size for the linear layer needs to be calculated based on the output dimensions of the concatenated tensors
        # For example, for 28x28 input images (as in MNIST), the size might be different
        self.fc = nn.Sequential(
            nn.Linear(16 * 3 * INPUT_SHAPE[-1] * INPUT_SHAPE[-2], 256),  # Replace <height> and <width> with the dimensions after the convolutional layers
            nn.ReLU(),
            nn.Linear(256, num_classes)  # Number of output classes
        )

    def forward(self, x):
        x.requires_grad_(True)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x = torch.cat((x1, x2, x3), dim=1)  # Concatenating along the channel dimension
        x = x.view(x.size(0), -1)  # Flatten the tensor for the fully connected layer
        x = self.fc(x)
        return x

# Get dataset

In [6]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST('.', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('.', train=False, download=True, transform=transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 55204703.70it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 25680664.37it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 76438960.57it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 3048084.60it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



# FLOP Count

In [7]:
def count_flops(model, input_size):
    inputs = torch.randn(input_size)
    flops = FlopCountAnalysis(model, inputs)
    return flop_count_table(flops)

inception_model = Inception(1, 10)
# Assuming the input size for CIFAR-10 (batch size, channels, height, width)
input_size = (1, 1, 28, 28)
flops_table = count_flops(inception_model, input_size)
print(flops_table)

| module                  | #parameters or shape   | #flops    |
|:------------------------|:-----------------------|:----------|
| model                   | 9.638M                 | 12.365M   |
|  conv1x1_3x3            |  1.184K                |  2.728M   |
|   conv1x1_3x3.0         |   16                   |   18.816K |
|    conv1x1_3x3.0.weight |    (8, 1, 1, 1)        |           |
|    conv1x1_3x3.0.bias   |    (8,)                |           |
|   conv1x1_3x3.1         |   1.168K               |   2.71M   |
|    conv1x1_3x3.1.weight |    (16, 8, 3, 3)       |           |
|    conv1x1_3x3.1.bias   |    (16,)               |           |
|  fc                     |  9.637M                |  9.636M   |
|   fc.0                  |   9.634M               |   9.634M  |
|    fc.0.weight          |    (256, 37632)        |           |
|    fc.0.bias            |    (256,)              |           |
|   fc.2                  |   2.57K                |   2.56K   |
|    fc.2.weight         

# Train and attribution functions

train and eval function

In [8]:
def train(epoch, model, train_loader, optimizer, criterion, device):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

    train_accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Train Loss = {train_loss / len(train_loader):.4f}, Train Accuracy = {train_accuracy:.2f}%")

    return train_accuracy

def test(epoch, model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

    test_accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Test Loss = {test_loss / len(test_loader):.4f}, Test Accuracy = {test_accuracy:.2f}%")

    return test_accuracy


functions for calculate attribution

In [9]:
def print_ig(test_loader, model, device):
    # Move the model to the device (CPU or CUDA)
    model.to(device)

    # Ensure the model is in evaluation mode
    model.eval()

    # Get a single batch from the test loader
    inputs, target_class = next(iter(test_loader))
    inputs = inputs.to(device)

    ig_attributions = {}

    # Iterate through each named module and compute attributions for Conv2d layers with learnable parameters
    for layer_name, layer_module in model.named_modules():
        # Check if the layer is a Conv2d layer with learnable parameters
        if isinstance(layer_module, nn.Conv2d) and any(p.requires_grad for p in layer_module.parameters(recurse=False)):
            # Initialize LayerIntegratedGradients for the layer
            lig = LayerIntegratedGradients(model, layer_module)

            # Compute the attributions for the current layer
            try:
                attributions = lig.attribute(inputs, target=target_class.to(device))
            except Exception as e:
                print(f"Error computing attributions for layer {layer_name}: {e}")
                continue

            # Print out the attributions for the current layer
            print(f'Layer: {layer_name}')
            print(f'Attribution: {attributions.cpu().detach().numpy().sum()}')

            # Store the sum of attributions in the dictionary
            ig_attributions[layer_name] = attributions.cpu().detach().numpy().sum()

            # Free up memory
            del attributions, lig

    return ig_attributions

# Usage example:
# ig_attributions = print_ig(test_loader, model, device)


In [10]:
def print_deeplift(test_loader, model, device):
    # Move the model to the specified device and set it to evaluation mode
    model.to(device).eval()

    # Get a batch of data from the loader
    inputs, target_class = next(iter(test_loader))
    inputs, target_class = inputs.to(device), target_class.to(device)

    dl_attributions = {}

    # Now compute the attributions for Conv2d layers
    for layer_name, layer_module in model.named_modules():
        # Skip the whole model's container and focus on Conv2d layers with learnable parameters
        if isinstance(layer_module, nn.Conv2d) and any(p.requires_grad for p in layer_module.parameters(recurse=False)):
            # Initialize LayerDeepLift with the current layer
            ldl = LayerDeepLift(model, layer_module)

            # Compute the attributions for the current layer
            '''try:
                attributions_ldl = ldl.attribute(inputs, target=target_class.to(device))
            except Exception as e:
                print(f"Error computing attributions for layer {layer_name}: {e}")
                continue'''
            attributions_ldl = ldl.attribute(inputs, target=target_class.to(device))


            # Print out the attributions for the current layer
            print(f'Layer: {layer_name}')
            print(attributions_ldl.cpu().data.numpy().sum())

            dl_attributions[layer_name] = attributions_ldl.cpu().data.numpy().sum()

            del attributions_ldl, ldl

    return dl_attributions

# Usage example:
# dl_attributions = print_deeplift(test_loader, model, device)


# Possible Hyperparameter grid search creation

in hyperparams_list_dict, each hyperparameter has corresponding possible choices as a list, during experiment, given hyperparams sequence, location each hyperperameter's location using hyperparameter encoding function to convert strings or classes back into their index in hyperparams_list_dict

In [11]:
'''sample_hyperparams_list_dict = {
    'initial_lr': [],
    'optimizer': torch.optim.Adam,  # Example optimizer
    'criterion': torch.nn.CrossEntropyLoss(),
    'train_data_used': 0.8,
    'train_set_shuffle': True,
    'train_batch_size': 64
}'''

"sample_hyperparams_list_dict = {\n    'initial_lr': [],\n    'optimizer': torch.optim.Adam,  # Example optimizer\n    'criterion': torch.nn.CrossEntropyLoss(),\n    'train_data_used': 0.8,\n    'train_set_shuffle': True,\n    'train_batch_size': 64\n}"

In [12]:
def generate_hyperparameter_combinations(hyperparams):
    """
    Generate a sequence of hyperparameter combinations.

    :param hyperparams: A dictionary where keys are the names of hyperparameters,
                        and values are lists of possible choices for each hyperparameter.
    :return: A list of dictionaries, each representing a unique combination of hyperparameters.
    """
    # Extract the hyperparameter names and their corresponding choices
    keys, values = zip(*hyperparams.items())

    # Generate all possible combinations of hyperparameter values
    all_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    return all_combinations

# Automated Experiments

In [13]:
def get_data_loader(hyperparams, train_dataset, test_dataset):
  shuffle = hyperparams['train_set_shuffle']
  train_batch_size = hyperparams["train_batch_size"]
  train_data_used_num = int(len(train_dataset) * hyperparams["train_data_used"])
  train_indices = torch.randperm(len(train_dataset))[:train_data_used_num]

  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=shuffle)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
  return train_loader, test_loader

In [14]:
def run_experiments(hyperparams, train_loader, test_loader, method, num_epochs, num_classes=NUM_CLASSES, in_channels=IN_CHANNELS):
    # Initialize model
    model = Inception(in_channels, num_classes).to(Training_Device)  # Assuming Inception is defined elsewhere

    # Use the optimizer from hyperparameters
    optimizer = hyperparams['optimizer'](model.parameters(), lr=hyperparams['initial_lr'])
    criterion = hyperparams['criterion']

    train_accuracy = {}
    test_accuracy = {}

    for epoch in range(num_epochs):
            # Assuming train and test functions are defined elsewhere
            train_acc = train(epoch, model, train_loader, optimizer, criterion, Training_Device)
            test_acc = test(epoch, model, test_loader, criterion, Training_Device)

            train_accuracy["train accuracy epoch"+str(epoch)] = train_acc
            test_accuracy["test accuracy epoch"+str(epoch)] = test_acc



    if method == "deeplift":
      dl_attributions = print_deeplift(test_loader, model, Feature_Attribution_Device)
      return dl_attributions, train_accuracy, test_accuracy
      print("\n")
    elif method == "integrated_gradients":
      ig_attributions = print_ig(test_loader, model, Feature_Attribution_Device)
      print("\n")
      return ig_attributions, train_accuracy, test_accuracy

# Example Usage
hyperparams = {
    'initial_lr': 0.001,
    'optimizer': torch.optim.Adam,  # Example optimizer
    'criterion': torch.nn.CrossEntropyLoss(),
    'train_data_used': 0.8,
    'train_set_shuffle': True,
    'train_batch_size': 64

}

#train_loader, test_loader = get_data_loader(hyperparams, train_dataset, test_dataset)
#print(run_experiments(num_epochs=1, hyperparams=hyperparams, train_loader=train_loader, test_loader=test_loader))

In [15]:
def run_experiments(hyperparams, train_loader, test_loader, num_epochs, num_classes=NUM_CLASSES, in_channels=IN_CHANNELS):
    # Initialize model
    model = Inception(in_channels, num_classes).to(Training_Device)  # Assuming Inception is defined elsewhere

    # Use the optimizer from hyperparameters
    optimizer = hyperparams['optimizer'](model.parameters(), lr=hyperparams['initial_lr'])
    criterion = hyperparams['criterion']

    train_accuracy = {}
    test_accuracy = {}

    for epoch in range(num_epochs):
            # Assuming train and test functions are defined elsewhere
            train_acc = train(epoch, model, train_loader, optimizer, criterion, Training_Device)
            test_acc = test(epoch, model, test_loader, criterion, Training_Device)

            train_accuracy["train accuracy epoch"+str(epoch)] = train_acc
            test_accuracy["test accuracy epoch"+str(epoch)] = test_acc



    print("deeplift")
    dl_attributions = print_deeplift(test_loader, model, Feature_Attribution_Device)
    print("integrated_gradient")
    ig_attributions = print_ig(test_loader, model, Feature_Attribution_Device)
    return ig_attributions, dl_attributions, train_accuracy, test_accuracy

# Example Usage
hyperparams = {
    'initial_lr': 0.001,
    'optimizer': torch.optim.Adam,  # Example optimizer
    'criterion': torch.nn.CrossEntropyLoss(),
    'train_data_used': 0.8,
    'train_set_shuffle': True,
    'train_batch_size': 64

}

#train_loader, test_loader = get_data_loader(hyperparams, train_dataset, test_dataset)
#print(run_experiments(num_epochs=1, hyperparams=hyperparams, train_loader=train_loader, test_loader=test_loader))

# Functions for saving attribution

In [16]:
def run_experiments_and_save(hyperparams_combinations, train_dataset, test_dataset, csv_file, num_run=NUM_RUN, num_epochs=NUM_EPOCHS):
    """
    Run experiments for each combination of hyperparameters, get feature layer attributions for DeepLift and Integrated Gradients,
    save the results to a CSV file after each run, and skip any combinations that have already been run.

    :param hyperparams_combinations: List of dictionaries with hyperparameter combinations.
    :param train_dataset: Training dataset.
    :param test_dataset: Test dataset.
    :param csv_file: Path to the CSV file for saving results.
    :param num_epochs: Number of epochs for training.
    """

    # Check if the CSV file exists and load existing data
    if os.path.exists(csv_file):
        existing_data = pd.read_csv(csv_file)
    else:
        existing_data = pd.DataFrame()

    for combo in hyperparams_combinations:
        print("\n\n")
        print("Combination:")
        print(combo)
        for i in range(num_run):  # For each run index
            print("runtime" + str(i))
            for method in ['deeplift', 'integrated_gradients']:  # For each method
                print(method)
                # Prepare data for checking if it's already processed
                combo_check = combo.copy()
                combo_check['method'] = method
                combo_check['run'] = i

                # Convert combo_check to string representation
                combo_check = {k: str(v) for k, v in combo_check.items()}



                if not existing_data.empty:
                    # Filter existing_data to only the columns of interest
                    filtered_data = existing_data[list(combo_check.keys())]
                    # Convert filtered_data to string representation
                    filtered_data = filtered_data.astype(str)
                    # Check if a row with the same combination exists
                    if any((filtered_data == pd.Series(combo_check)).all(axis=1)):
                        print("Combination already processed, skipping...")
                        continue  # Skip if combination is already processed

                # Set seed for reproducibility
                random.seed(i)
                np.random.seed(i)
                torch.manual_seed(i)
                if torch.cuda.is_available():
                    torch.cuda.manual_seed_all(i)

                train_loader, test_loader = get_data_loader(combo, train_dataset, test_dataset)

                # Run experiments and compute attributions
                attr = run_experiments(hyperparams=combo, train_loader=train_loader, test_loader=test_loader, method=method, num_epochs=num_epochs)

                # Prepare data for saving
                combo_results = combo.copy()
                for d in attr:
                    combo_results.update(d)

                combo_results['method'] = method
                combo_results['run'] = i

                # Convert all values in combo_results to strings
                combo_results = {k: str(v) for k, v in combo_results.items()}

                # Append results to the existing data
                existing_data = existing_data.append(combo_results, ignore_index=True)

                # Save the data to CSV after each run
                existing_data.to_csv(csv_file, index=False)

In [17]:
def run_experiments_and_save(hyperparams_combinations, train_dataset, test_dataset, csv_file, num_run=NUM_RUN, num_epochs=NUM_EPOCHS):
    """
    Run experiments for each combination of hyperparameters, get feature layer attributions for DeepLift and Integrated Gradients,
    save the results to a CSV file after each run, and skip any combinations that have already been run.

    :param hyperparams_combinations: List of dictionaries with hyperparameter combinations.
    :param train_dataset: Training dataset.
    :param test_dataset: Test dataset.
    :param csv_file: Path to the CSV file for saving results.
    :param num_epochs: Number of epochs for training.
    """

    # Check if the CSV file exists and load existing data
    if os.path.exists(csv_file):
        existing_data = pd.read_csv(csv_file)
    else:
        existing_data = pd.DataFrame()

    for combo in hyperparams_combinations:
        print("\n\n")
        print("Combination:")
        print(combo)
        for i in range(num_run):  # For each run index
            print("runtime" + str(i))
            # Prepare data for checking if it's already processed
            combo_check = combo.copy()
            combo_check['run'] = i

            # Convert combo_check to string representation
            combo_check = {k: str(v) for k, v in combo_check.items()}



            if not existing_data.empty:
                # Filter existing_data to only the columns of interest
                filtered_data = existing_data[list(combo_check.keys())]
                # Convert filtered_data to string representation
                filtered_data = filtered_data.astype(str)
                # Check if a row with the same combination exists
                if any((filtered_data == pd.Series(combo_check)).all(axis=1)):
                    print("Combination already processed, skipping...")
                    continue  # Skip if combination is already processed

                # Set seed for reproducibility
            random.seed(i)
            np.random.seed(i)
            torch.manual_seed(i)
            if torch.cuda.is_available():
                torch.cuda.manual_seed_all(i)

            train_loader, test_loader = get_data_loader(combo, train_dataset, test_dataset)

                # Run experiments and compute attributions
            ig_attributions, dl_attributions, train_accuracy, test_accuracy = run_experiments(hyperparams=combo, train_loader=train_loader, test_loader=test_loader, num_epochs=num_epochs)
            for method in ['deeplift', 'integrated_gradients']:
              if method == 'deeplift':
                  attr = dl_attributions, train_accuracy, test_accuracy
              elif method == 'integrated_gradients':
                  attr = ig_attributions, train_accuracy, test_accuracy

              combo_results = combo.copy()
              for d in attr:
                  combo_results.update(d)

              combo_results['method'] = method
              combo_results['run'] = i

              # Convert all values in combo_results to strings
              combo_results = {k: str(v) for k, v in combo_results.items()}

              # Append results to the existing data
              existing_data = existing_data.append(combo_results, ignore_index=True)

            # Save the data to CSV after each run
            existing_data.to_csv(csv_file, index=False)

# Experiments

In [18]:
hyperparams_choice_list = {
    'initial_lr': [0.001, 0.0007, 0.0003],
    'optimizer': [torch.optim.Adam], # Example optimizer
    'criterion': [torch.nn.CrossEntropyLoss()],
    'train_data_used': [1],
    'train_set_shuffle': [True,False],
    'train_batch_size': [64, 32, 16]
}

combinations = generate_hyperparameter_combinations(hyperparams_choice_list)
for combo in combinations:
    print(combo)

{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 64}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 32}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 16}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': False, 'train_batch_size': 64}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': False, 'train_batch_size': 32}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'trai

In [19]:
file_path=os.path.join('/content/drive/My Drive/2023 InterpretingNN/code/experiment and result stage2/Baseline Mnist', 'Baseline Mnist 3*3.csv')
print(file_path)

/content/drive/My Drive/2023 InterpretingNN/code/experiment and result stage2/Baseline Mnist/Baseline Mnist 3*3.csv


In [ ]:
run_experiments_and_save(combinations, train_dataset, test_dataset, csv_file=file_path)




Combination:
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 64}
runtime0
Epoch 0: Train Loss = 0.2871, Train Accuracy = 92.58%
Epoch 0: Test Loss = 0.1276, Test Accuracy = 95.93%
deeplift
Layer: conv1x1_3x3.0
285.70093
Layer: conv1x1_3x3.1
285.70093
integrated_gradient
Layer: conv1x1_3x3.0
Attribution: 857.0340103410758
Layer: conv1x1_3x3.1
Attribution: 857.0340092389647
runtime1


<ipython-input-17-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-17-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 0.2659, Train Accuracy = 92.84%
Epoch 0: Test Loss = 0.1186, Test Accuracy = 96.30%
deeplift
Layer: conv1x1_3x3.0
292.2001
Layer: conv1x1_3x3.1
292.20016
integrated_gradient
Layer: conv1x1_3x3.0
Attribution: 875.4536476521175
Layer: conv1x1_3x3.1
Attribution: 875.4536271831612
runtime2


<ipython-input-17-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-17-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 0.2967, Train Accuracy = 92.16%
Epoch 0: Test Loss = 0.1268, Test Accuracy = 95.97%
deeplift
Layer: conv1x1_3x3.0
363.74313
Layer: conv1x1_3x3.1
363.74323
integrated_gradient
Layer: conv1x1_3x3.0
Attribution: 1092.483618710605
Layer: conv1x1_3x3.1
Attribution: 1092.4836212260852
runtime3


...

<ipython-input-17-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-17-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 0.2336, Train Accuracy = 93.17%
Epoch 0: Test Loss = 0.1747, Test Accuracy = 94.56%
deeplift
Layer: conv1x1_3x3.0
410.59097
Layer: conv1x1_3x3.1
410.591
integrated_gradient
Layer: conv1x1_3x3.0
Attribution: 1231.6536852332574
Layer: conv1x1_3x3.1
Attribution: 1231.6536815832533
runtime14


<ipython-input-17-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-17-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 0.2603, Train Accuracy = 92.36%
Epoch 0: Test Loss = 0.1427, Test Accuracy = 95.39%
deeplift
Layer: conv1x1_3x3.0
790.7488
Layer: conv1x1_3x3.1
790.7489
integrated_gradient
Layer: conv1x1_3x3.0
Attribution: 2378.1777737958846
Layer: conv1x1_3x3.1
Attribution: 2378.1777431929336


<ipython-input-17-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-17-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
